## **Google Advanced Data Analytics**

In [28]:
# Operational packages
import pandas as pd
import numpy as np
import random
from scipy import stats
import pickle

# Visualisation packages
import matplotlib.pyplot as plt
import seaborn as sns

# Statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, PredefinedSplit
import sklearn.metrics as metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LogisticRegression

### **Data validation:**
```
df.describe(include='all')
df.info()
df['column'].value_counts()
```

### **Confidence interval:**
```
stats.norm.interval(alpha=0.95, loc=sample_mean, scale=sample_std_error)
sample_std_error = sample_sd / np.sqrt(n)
```

### **Hypothesis testing:**

Assuming normally distributed populations<br>
If population SD is known, use z-test. If unknown, use t-test (fatter tails)<br>
Note: if we are looking at proportions, must use z-test<br>

**2 sample t-test (A/B testing):**<br>
```
p_value = stats.ttest_ind(a, b, equal_var, alternative)[1]
equal_var: {True, False}
alternative : {'two-sided', 'less', 'greater'}
```

**1 sample t-test:**<br>
```
p_value = stats.ttest_1samp(a, popmean, alternative)[1]
```

### **Linear regression:**

To model a continuous variable (estimate mean of y)

Key assumptions:
- Linearity
- Normal distribution of residuals
- Independent observations
- Homoscedasticity
- For multiple regression: no multicollinearity between independent variables (low VIF)

**Check scatter plots of pairs:**
```
sns.pairplot(df)
```

**Run regression:**
```
ols_formula = 'y_col ~ x1_col + x2_col + C(x_categorical_col)'
OLS = ols(formula=ols_formula, data=df)
model = OLS.fit()
model.summary()
fitted_values = model.fittedvalues
residuals = model.resid
sns.regplot(x=x_col, y=y_col, data=df)
```

**Check homoscedasticity (random cloud of fitted vs residuals):**
```
sns.scatterplot(fitted_values, residuals);
```

**Check normal distribution of residuals:**
```
sns.histplot(residuals);
sm.qqplot(residuals, line='s');
```

**Check VIF (lower is better, minimum is 1):**
```
vif = [variance_inflation_factor(df_X.values, i) for i in range(df_X.shape[1])]
df_vif = pd.DataFrame(vif, index=df_X.columns, columns=['VIF'])
```

### **Analysis of Variance:**

**ANOVA:**<br>
- Test the difference of means between >=3 groups<br>
- Extension of t-tests<br>
- Applied to results of linear regression model<br>
- One-way ANOVA: compares the means of 1 continuous dependent variable in >=3 groups of 1 categorical variable. (H0: all means are equal)<br>
- Two-way ANOVA: compares the means of 1 continuous dependent variable in >=3 groups of 2 categorical variables. (H0: both categories + their interaction all have no impact on mean)<br>
- If PR(>F) is small, reject the null hypothesis
```
model = ols(formula=ols_formula, data=df).fit()
sm.stats.anova_lm(model, typ=2)
```
ANOVA post-hoc test (Tukey's HSD test):<br>
Pairwise comparison between all available groups while controlling for the error rate<br>
Trying to isolate which group is different
```
tukey_oneway = pairwise_tukeyhsd(endog=df_y, groups=df_X, alpha=0.05)
tukey_oneway.summary()
```

**ANCOVA:**<br>
Test the difference of means between >=3 groups, while controlling for the effects of covariates (aka variables irrelevant to your test)<br>

**MANOVA:**<br>
Multi-variate analysis of variance<br>
Compare how >=2 continuous outcome variables vary according to categorical independent variables<br>
One-way: 1 categorical independent variable<br>
Two-way: 2 categorical independent variables<br>
H0: means of both continuous outcome variables are not impacted by the categorical independent variables

**MANCOVA:**<br>
MANOVA while controlling for the effects of covariates

### **Bias vs Variance:**

High bias = simple / underfit<br>
High variance = complex / overfit<br>

**Regularised regression**
Shrinks coefficients towards 0
- Lasso: irrelevant coefficients are set to 0
- Ridge: irrelevant coefficients shrink towards 0, but don't reach 0
- Elastic net: combination of Lasso and Ridge

### **Logistic regression:**

To model a categorical dependent variable Y<br>
Binomial logistic regression: Y is binary<br>

Logit is the most common link function used to linearly relate the X variables to the probability of Y.

Key assumptions:
- Linearity: Linear relationship between each X variable and logit of P(Y=1)
- Independent observations (can multiply probabilities)
- No multicollinearity between independent variables
- No extreme outliers

logit(p) = log(p / (1-p)) = B0 + B1X1 + ... + BnXn -> Use MLE to find the betas
```
clf = LogisticRegression().fit(X_train, y_train)
print(clf.coef_, clf.intercept_)
y_pred = clf.predict(X_test)
clf.predict_proba(X_test)[::,-1]
sns.regplot(x=x_col, y=y_col, data=df, logistic=True)
```
**Confusion matrix**
```
cm = metrics.confusion_matrix(y_test, y_pred, labels=clf.classes_)
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
```
**Precision, recall and accuracy**

- Precision: Proportion of positive predictions that were true positives
- Recall: Proportion of positives that the model was able to identify correctly
- Accuracy: Proportion of data points that were correctly categorised
- F1 (harmonic mean): 2 * precision * recall / (precision + recall)
```
print(metrics.precision_score(y_test, y_pred))
print(metrics.recall_score(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.f1_score(y_test, y_pred))
```
ROC curve and AUC

### **Data preparation:**

**Train-test split:**
```
df_X = df[[col_X1, col_X2, col_X3]]
df_y = df[[col_y]]
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, stratify=df_y, test_size=0.3)
```
**Get dummies:**
```
df = pd.get_dummies(df, drop_first=True)
```
**Scaling data:**
```
scaler = MinMaxScaler()   # or StandardScaler() or Normalizer()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
```
**To export/import a fitted model:**
```
with open(path + 'filename.pickle', 'wb') as to_write:
    pickle.dump(model, to_write)

with open(path + 'filename.pickle', 'rb') as to_read:
    model = pickle.load(to_read)
```

### **Naive Bayes:**

Supervised | Classification
- Assumption of independence among predictors
- Does not require data scaling

```
from sklearn.naive_bayes import GaussianNB      # for continuous, normally distributed features
from sklearn.naive_bayes import MultinomialNB   # for discrete features
from sklearn.naive_bayes import BernoulliNB     # for Boolean features
from sklearn.naive_bayes import CategoricalNB   # for categorical features

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
```

### **K-Means:**

Unsupervised | Classification (Partitioning)

- Works better with scaled data

4 steps:
1. Initiate k centroids (choose k)
2. Assign each data point to its nearest centroid
3. Recalculate the centroid of each cluster
4. Repeat step 2 and 3 until convergence
```
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=k).fit(df)
print(kmeans.labels_.shape)
print(kmeans.labels_)
print(np.unique(kmeans.labels_))
print(kmeans.cluster_centers_)
print(kms.inertia_)
print(silhouette_score(df_X, kms.labels_)
```
**Inertia**: sum of squared distances between each observation and its nearest centroid<br>
Elbow method: plot k values vs inertia, and choose highest k with a meaningful drop in inertia

**Silhouette score**: mean of silhouette coefficients for all observations. Accounts for intra- and inter-cluster<br>
-1 <= s <= 1<br>
Want k to maximise silhouette score

### **Decision Trees:**

Supervised | Classification

- Require no assumptions regarding the distribution of underlying data
- Can handle collinearity
- Little preprocessing required
- Susceptibile to overfitting

Each split is determined by which variables and cut-off values offer the most predictive power
```
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)
dt_pred = decision_tree.predict(X_test)

plt.figure(figsize=(15,12))
plot_tree(decision_tree, max_depth=2, fontsize=14, feature_names=df_X.columns, filled=True);
plt.show()

```
**Gini impurity:** how pure the node is, from 0 to 0.5.<br>
0: the node is a leaf<br>
0.5: all classes are equally represented in the node<br>

**Hyperparameters:**
- max_depth
- min_samples_split
- min_samples_leaf

Tuning hyperparameters with gridsearch:
```
tuned_decision_tree = DecisionTreeClassifier()
tree_para = {'max_depth':[4,5,6,7,8,9,10,11,12,15,20,30,40,50],
             'min_samples_leaf': [2, 5, 10, 20, 50]}
scoring = {'accuracy', 'precision', 'recall', 'f1'}
clf = GridSearchCV(tuned_decision_tree, tree_para, scoring=scoring, cv=5, refit='f1')
clf.fit(X_train, y_train)
print(clf.best_estimator_)
print(clf.best_score_)
```

### **Random Forest:**

Supervised | Classification or Regression | Ensemble

**Bagging:**
- AKA Bootstrap Aggregating
- Bootstrapping = sampling with replacement
- Each individual model is called a base learner
- Each base learner is trained on a unique random subset of the training data

**Random Forest:**
- Bagging based on decision trees
- Each tree uses a random subset of the available features (so not all features)
- Reduces variance, but not bias

**Hyperparameters:**
- All the same as decision trees, plus:
- max_features
- n_estimators
- max_samples
```
from sklearn.ensemble import RandomForestClassifier   # for classification tasks
from sklearn.ensemble import RandomForestRegressor    # for regression tasks
rf = RandomForestClassifier()
rf_cv = GridSearchCV(rf, cv_params, scoring=scoring, cv=5, refit='f1')
```

### **Boosting:**

Supervised | Classification or Regression | Ensemble

- Builds an ensemble of weak learners sequentially
- Each consecutive base learner tries to correct the errors of the one before

**AdaBoost:**
- AKA Adaptive Boosting
- Tree-based boosting methodology
- Each consecutive base learner assigns greater weight to the observations incorrectly predicted by the preceding learner
- Final prediction is based on weighted vote (classification) or weighted mean prediction (regression)
- Runs slowly, because the base learners can't be trained in parallel

Advantages:
- Reduces variance and bias
- Easy to interpret
- Doesn't require data to be scaled or normalised
- Can handle multicollinearity
- Can use numeric or categorical inputs
- Robust to outliers

**Gradient Boosting (GBMs):**
- Each base learner in the sequence is built to predict the residual errors of the model that preceded it
- Tree-based
```
from xgboost import XGBClassifier   # For classification tasks
from xgboost import XGBRegressor   # For regression tasks
from xgboost import plot_importance
xgb = XGBClassifier(objective='binary:logistic')
xgb_cv = GridSearchCV(xgb, cv_params, scoring=scoring, cv=5, refit='f1')
xgb_cv.fit(X_train, y_train)
```
Advantages:
- High accuracy
- Scalable
- Work well with missing data
- Usual advantages of tree-based methods

Disadvantages:
- Black-box model
- Lots of hyperparameters
- Prone to overfitting
- Doesn't extrapolate well to new data

Hyperparameters:
- max_depth
- n_estimators
- learning_rate
- min_child_weight
- colsample_bytree
- subsample